In [4]:
from riix.models.elo import Elo
from riix.utils import TimedPairDataset, split_matchup_dataset, generate_matchup_data
from riix.metrics import binary_metrics_suite

In [5]:
df = generate_matchup_data() # replace with your **polars** dataframe
print(df.schema)
full_dataset = TimedPairDataset(
    df=df,
    competitor_cols=['competitor_1', 'competitor_2'],
    outcome_col='outcome',
    datetime_col='date',
    rating_period='1D',
)
train_dataset, test_dataset = split_matchup_dataset(full_dataset, test_fraction=0.2)
print(f'{len(train_dataset)=}, {len(test_dataset)=}')

Schema({'competitor_1': String, 'competitor_2': String, 'outcome': Float64, 'date': Datetime(time_unit='ms', time_zone=None)})
loaded dataset with:
10000 matchups
100 unique competitors
10 rating periods of length 1D
split into train_dataset of length 8000 and test_dataset of length 2000
len(train_dataset)=8000, len(test_dataset)=2000


In [6]:
model = Elo(competitors=full_dataset.competitors)
model.fit_dataset(train_dataset)
test_probs = model.fit_dataset(test_dataset, return_pre_match_probs=True)
test_metrics = binary_metrics_suite(probs=test_probs, outcomes=test_dataset.outcomes)
print(test_metrics)

{'accuracy': np.float64(0.72975), 'accuracy_without_draws': np.float64(0.730095142714071), 'log_loss': np.float64(0.5359083106524117), 'brier_score': np.float64(0.1793377446861956)}


In [7]:
model.print_leaderboard(num_places=5)

competitor   	rating
competitor_69	1874.170019
competitor_75	1827.933570
competitor_12	1826.119826
competitor_81	1825.071605
competitor_30	1802.338983
